<a href="https://colab.research.google.com/github/sameehaafr/sigai-tsf/blob/master/lstm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive  
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from numpy import array

In [ ]:
la_2020 = pd.read_csv('/content/gdrive/MyDrive/SIGAI TSF/DATA/LA_pm10_2020.csv')
la_2021 = pd.read_csv('/content/gdrive/MyDrive/SIGAI TSF/DATA/LA_pm10_2021.csv')
la_2022 = pd.read_csv('/content/gdrive/MyDrive/SIGAI TSF/DATA/LA_pm10_2022.csv')

In [ ]:
# subsetting
la_2020 = la_2020[['Date', 'Daily Mean PM10 Concentration']]
la_2021 = la_2021[['Date', 'Daily Mean PM10 Concentration']]
la_2022 = la_2022[['Date', 'Daily Mean PM10 Concentration']]

In [ ]:
# converting all to datetime
la_2020['Date'] = pd.to_datetime(la_2020['Date'])
la_2021['Date'] = pd.to_datetime(la_2021['Date'])
la_2022['Date'] = pd.to_datetime(la_2022['Date'])

<ipython-input-5-22adbb8af597>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_2020['Date'] = pd.to_datetime(la_2020['Date'])
<ipython-input-5-22adbb8af597>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_2021['Date'] = pd.to_datetime(la_2021['Date'])


In [ ]:
# merging
la_pm10 = pd.concat([la_2020, la_2021, la_2022])
la_pm10.rename(columns = {'Daily Mean PM10 Concentration':'daily_pm10'}, inplace = True)

In [ ]:
#merging
la_pm10 = pd.concat([la_2020, la_2021])
la_pm10.rename(columns = {'Daily Mean PM10 Concentration':'daily_pm10'}, inplace = True)
la_2022.rename(columns = {'Daily Mean PM10 Concentration':'daily_pm10'}, inplace = True)

In [ ]:
la_pm10.head()

,Date,daily_pm10
0,2020-01-04,25
1,2020-01-10,20
2,2020-01-16,32
3,2020-01-22,21
4,2020-01-28,23


In [ ]:
la_pm10.tail()

,Date,daily_pm10
983,2021-12-27,6
984,2021-12-28,6
985,2021-12-29,5
986,2021-12-30,4
987,2021-12-31,7


In [ ]:
# finding row with max pm10 value
la_pm10.loc[(la_pm10['daily_pm10'] == max(la_pm10['daily_pm10']))]

,Date,daily_pm10
906,2021-10-11,411


In [ ]:
#normalizing
la_pm10['daily_pm10'] = (la_pm10['daily_pm10'] - la_pm10['daily_pm10'].mean()) / la_pm10['daily_pm10'].std()

In [ ]:
la_pm10

,Date,daily_pm10
0,2020-01-04,-0.159737
1,2020-01-10,-0.422918
2,2020-01-16,0.208717
3,2020-01-22,-0.370282
4,2020-01-28,-0.265009
...,...,...
983,2021-12-27,-1.159826
984,2021-12-28,-1.159826
985,2021-12-29,-1.212462
986,2021-12-30,-1.265098


Data Split

In [ ]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
# define input sequence
daily_pm10 = la_pm10['daily_pm10'].values.tolist()

In [ ]:
daily_pm10 = daily_pm10[0:len(daily_pm10)-1]

In [ ]:
x_train, y_train = split_sequence(daily_pm10, 10)

In [ ]:
x_train

array([[-0.15973694, -0.42291816,  0.20871675, ...,  0.20871675,
        -0.05446446, -0.4755544 ],
       [-0.42291816,  0.20871675, -0.37028191, ..., -0.05446446,
        -0.4755544 ,  0.36662548],
       [ 0.20871675, -0.37028191, -0.26500943, ..., -0.4755544 ,
         0.36662548, -1.21246179],
       ...,
       [-0.4755544 , -0.26500943,  0.10344427, ..., -1.21246179,
        -1.26509804, -1.15982555],
       [-0.26500943,  0.10344427,  0.261353  , ..., -1.26509804,
        -1.15982555, -1.15982555],
       [ 0.10344427,  0.261353  ,  0.10344427, ..., -1.15982555,
        -1.15982555, -1.21246179]])

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_train.shape

(1918, 10, 1)

In [ ]:
# define model
n_steps = 10
n_features = 1
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [ ]:
model.fit(x_train, y_train, epochs=200, verbose=0)

In [25]:
import joblib
from joblib import Parallel, delayed

joblib.dump(model, '/content/gdrive/MyDrive/SIGAI TSF/models/keras_lstm_1.pkl')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-02-17 18:23:40         1788
metadata.json                                  2023-02-17 18:23:40           64
variables.h5                                   2023-02-17 18:23:41       145576


['/content/gdrive/MyDrive/SIGAI TSF/models/keras_lstm_1.pkl']